In [70]:
import requests
import json
import pandas as pd

URL = "https://v3.football.api-sports.io/"
endpoint = "leagues"

headers = {
	'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "fb2140228973d644db847895c454c22b"
}

params = {}

def api_req(endpoint, params):
    response = requests.get(
        URL+endpoint,
        headers = headers,
        params = params
    )
    if response.status_code == 200:
            
        remaining = response.headers.get("x-ratelimit-requests-remaining")
        data = response.json()
        print(f"requests before reaching limit {remaining}")

    else:
        print(f"Error {response.status_code}, {response.text}")

    return data

In [71]:
def get_cols(response, coverage):
    coverages = response
    for key, val in coverages.items():
        if isinstance(val, dict):
            coverage = get_cols(val, coverage)
        else:
            coverage.append(key)
    return coverage

In [72]:
# functions inserting data into contests and seasons dfs
def insert_contests(response):
    contests_rows = []
    for league in response['response']:
        contest_row = {'league_id': league['league']['id'],
                       'name': league['league']['name'],
                       'type': league['league']['type'],
                       'logo': league['league']['logo'],
                       'country': league['country']['name'],
                       'country_code': league['country']['code'],
                       'seasons': len(league['seasons'])}
        contests_rows.append(contest_row)
    contests = pd.DataFrame(contests_rows)
    return contests

def insert_seasons(response):
    season_rows = []
    for league in response['response']:
        for season in league['seasons']:
            row = {}
            row['league_id'] = league['league']['id']
            for key, val in season.items():

                if key == 'coverage':
                    for k, v in season['coverage'].items():
                        if k == 'fixtures':
                            for a, b in season['coverage']['fixtures'].items():
                                row[a] = b
                        else:
                            row[k] = v
                else:
                    row[key] = val
            season_rows.append(row)
    
    seasons = pd.DataFrame(season_rows)
    return seasons

In [73]:
leagues = api_req(endpoint,params)

requests before reaching limit 7496


In [74]:
covs = []
coverages = get_cols(leagues['response'][0]['seasons'][0], covs)

In [75]:
covs

['year',
 'start',
 'end',
 'current',
 'events',
 'lineups',
 'statistics_fixtures',
 'statistics_players',
 'standings',
 'players',
 'top_scorers',
 'top_assists',
 'top_cards',
 'injuries',
 'predictions',
 'odds']

In [82]:
contests = insert_contests(leagues)
seasons = insert_seasons(leagues)

In [77]:
contests.set_index('league_id', inplace=True)

In [86]:
contests.head()

,league_id,name,type,logo,country,country_code,seasons
2,61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,15
3,144,Jupiler Pro League,League,https://media.api-sports.io/football/leagues/1...,Belgium,BE,15
5,39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,15
6,78,Bundesliga,League,https://media.api-sports.io/football/leagues/7...,Germany,DE,14
7,135,Serie A,League,https://media.api-sports.io/football/leagues/1...,Italy,IT,14


In [87]:
uefa = pd.read_csv('data/UEFA_Ranking_2024.csv')
eur_cs = list(uefa['country'].unique())
contests = contests[contests['country'].isin(eur_cs)]
eur_ss = list(contests['league_id'].unique())
eur_seasons = seasons[seasons['league_id'].isin(eur_ss)]
eur_seasons = eur_seasons.merge(contests[['league_id','country','name']], on='league_id')
eur_seasons = eur_seasons[[
    'league_id', 'name', 'country', 
    'year', 'players','statistics_fixtures', 
    'current', 'predictions', 'odds'
]]

In [17]:
es = pd.read_csv('data/european_seasons.csv')

,Unnamed: 0,league_id,name,country,year,players,statistics_fixtures,current,predictions,odds
0,0,39,Premier League,England,2010,True,False,False,True,False
1,1,39,Premier League,England,2011,True,False,False,True,False
2,2,39,Premier League,England,2012,True,False,False,True,False
3,3,39,Premier League,England,2013,True,False,False,True,False
4,4,39,Premier League,England,2014,True,True,False,True,False


In [107]:
diff_df = eur_seasons.merge(es[['league_id','year']], on=['league_id','year'], how='left', indicator=True)
only_in_df2 = diff_df[diff_df['_merge'] == 'left_only'].drop(columns='_merge')

In [113]:
new_seasons = list(zip(only_in_df2['league_id'],only_in_df2['year']))

In [119]:
with open('data/new_seasons.csv', 'w') as file:
    json.dump(new_seasons, file)